In [1]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['BASE_PATH'] = '..'

In [3]:
import hydra
import os
import os.path as osp

import torch
from torch import FloatTensor, Tensor, LongTensor

from omegaconf import DictConfig, OmegaConf
from hydra.utils import instantiate

from typing import Dict, List, Optional, Union, Tuple

from lightning import seed_everything, Trainer
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.strategies.ddp import DDPStrategy
from glob import glob
from torch_ema import ExponentialMovingAverage

from torch.utils.data import DataLoader

from diffusion import Config
import diffusion
import json

In [4]:
from diffusion.dynamics import SDE, RSDE, EulerSolver
from diffusion.utils import calc_model_grads_norm, calc_model_weights_norm, filter_losses
from diffusion.models.contextual_denoising.modeling_clean_encoder import T5EncoderModel
from diffusion.models.contextual_denoising.modeling_noisy_encoder import BertLMHeadModel
from diffusion.models.contextual_denoising.score_estimator import ScoreEstimator
from diffusion.models.contextual_denoising.typings import EncoderOutput

from diffusion.helper import LinearWarmupLR
from diffusion.dataset import EncNormalizer, enc_normalizer


from functools import partial
from tqdm.auto import tqdm, trange

In [5]:
def load_model(exp_folder: str, ckpt_num: int, use_ema: bool = True):
    seed_everything(1337, workers=True)

    cfg = OmegaConf.load(osp.join(exp_folder, 'config.yaml'))
    cfg.lightning_wrapper.sde_cfg.ode_sampling = True
    cfg.datamodule.valid_dataloader_cfg.batch_size = 512
    yaml_cfg = OmegaConf.to_yaml(cfg)
    #print(yaml_cfg)
    print(osp.abspath('.'))

    wrapped_model = instantiate(cfg.lightning_wrapper, _recursive_=False)
    ckpt_path = osp.join(exp_folder, f'step_{ckpt_num}.ckpt')
    print(f'ckpt_path={ckpt_path}')
    ckpt = torch.load(
        ckpt_path,
        map_location='cpu'
    )
    wrapped_model.load_state_dict(
        ckpt['state_dict'],
        strict=True
    )
    prefix_folder = 'ema_' if use_ema else ''
    if use_ema:
        from torch_ema import ExponentialMovingAverage
        ema = ExponentialMovingAverage(wrapped_model.parameters(), 0)
        ema.load_state_dict(
            ckpt['callbacks']['EMACallback']
        )
        ema.copy_to(wrapped_model.parameters())
    #wrapped_model.score_estimator.load_state_dict(
    #    torch.load('score_estimator.pth', map_location='cpu')
    #)
    wrapped_model.eval()
    return wrapped_model, cfg

In [6]:
wrapped_model, cfg = load_model('../experiments/wiki-pretrain-nam-noisy-067-bs512-t2', ckpt_num=1_000_000, use_ema=True)
cfg: diffusion.Config

[rank: 0] Global seed set to 1337


/home/tbadmaev/cls_glue_diff/light_diffusion/notebooks


If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['enc_normalizer.enc_mean', 'enc_normalizer.enc_std']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference

RESTORED SLAVYAN
ckpt_path=../experiments/wiki-pretrain-nam-noisy-067-bs512-t2/step_1000000.ckpt


In [7]:
print(json.dumps(OmegaConf.to_container(cfg), indent=4))

{
    "max_steps": 1000000,
    "seed": 0,
    "every_n_train_steps": 50000,
    "grad_clip_norm": 1.0,
    "project": "cross_attention",
    "exp_name": "wiki-pretrain-nam-noisy-067-bs512-t2",
    "pretrained_path": null,
    "resume_path": null,
    "max_epochs": 50,
    "every_n_epochs": 1,
    "precision": "bf16-mixed",
    "lightning_wrapper": {
        "optim_partial": {
            "_target_": "torch.optim.AdamW",
            "_partial_": true,
            "lr": 0.0002,
            "weight_decay": 0.01,
            "betas": [
                0.9,
                0.98
            ],
            "eps": 1e-06
        },
        "sched_partial": {
            "_target_": "diffusion.LinearWarmupLR",
            "_partial_": true,
            "warmup_steps": 5000,
            "warmup_start_lr": 1e-06
        },
        "noisy_enc_normalizer_cfg": {
            "_target_": "diffusion.EncNormalizer",
            "enc_mean_path": "wiki_pret_old/encodings-bert_base-wiki-mean.pt",
        

In [8]:
datamodule: diffusion.SimpleDataModule = instantiate(cfg.datamodule, _recursive_=False)
datamodule.setup()

/home/tbadmaev/new_conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Found cached dataset parquet (/home/tbadmaev/.cache/huggingface/datasets/Graphcore___parquet/Graphcore--wikipedia-bert-128-d489528ddee484b2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached sp

In [9]:
loader: DataLoader = datamodule.val_dataloader()[0]
loader

/home/tbadmaev/new_conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device
iterator_loader = iter(loader)
batch = next(iterator_loader)
batch.keys()

/home/tbadmaev/new_conda/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


dict_keys(['clean_input_ids', 'clean_attention_mask', 'noisy_input_ids', 'noisy_token_type_ids', 'noisy_attention_mask'])

In [11]:
from diffusion.utils import dict_to_device

In [12]:
batch = dict_to_device(batch, device)
wrapped_model.to(device)
latent_ode_encs = torch.load('latent_ode_encs.pth', map_location='cpu').to(device)

SlavaContextualDenoising(
  (noisy_part_encoder): BertLMHeadModel(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_feat

In [13]:
@torch.no_grad()
def ode_cycle_forward(batch: Dict[str, Tensor]):
    wrapped_model.eval()
    assert wrapped_model.solver.rsde.ode_sampling is True, "Not ODE sampling"
    
    to_clean_part, to_noise_part = wrapped_model.split_batch(batch)
    clean_part: EncoderOutput = wrapped_model.clean_part_encoder.forward(**to_clean_part)
    noisy_part: EncoderOutput = wrapped_model.noisy_part_encoder.forward(**to_noise_part)
    
    noisy_part_attention_mask = torch.ones_like(
        batch['noisy_attention_mask']
    )
    
    shape = noisy_part_attention_mask.shape + (clean_part.normed.shape[-1],)
    cross_encodings = clean_part.normed
    
    target_encodings = noisy_part.normed
    
    cross_attention_mask = to_clean_part['attention_mask']
    attn_mask = noisy_part_attention_mask

    score_call = partial(
        wrapped_model.score_estimator.forward,
        cross_attention_mask=cross_attention_mask,
        cross_encodings=cross_encodings
    )
    verbose = True
    with torch.no_grad():
        x_t = target_encodings

        timesteps = torch.linspace(
            wrapped_model.sde.T / wrapped_model.sde.N,
            wrapped_model.sde.T,
            wrapped_model.sde.N,
            device=device
        )
        rang = trange if verbose else range

        for idx in rang(wrapped_model.sde.N):
            t = timesteps[idx]
            input_t = t * torch.ones(shape[0], device=device)
            
            dt = 1. / wrapped_model.sde.N
            
            rsde_params = wrapped_model.solver.rsde.sde(score_call, x_t, input_t, attn_mask)
            
            drift = rsde_params['drift']
            # return rsde_params
            
            x_t = x_t + drift * dt
    return x_t

In [ ]:
latent_ode_encs = ode_cycle_forward(batch)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
latent_ode_encs

In [ ]:
torch.save(latent_ode_encs.cpu(), 'latent_ode_encs.pth')

In [ ]:
import torch
import numpy as np

from matplotlib import pyplot as plt

In [ ]:
latent_ode_encs = torch.load('latent_ode_encs.pth', map_location='cpu')

In [ ]:
latent_ode_encs = latent_ode_encs.cpu().numpy()
latent_ode_encs.shape

In [ ]:
def plot_coord_distrib(coord_idx: int):
    obs = latent_ode_encs.reshape(-1, 768)[:, coord_idx]
    plt.figure(figsize=(12, 8))
    counts, bins = np.histogram(obs, bins=100)
    plt.stairs(counts / obs.shape[0], bins)

In [ ]:
plot_coord_distrib(13)